In [71]:
import numpy as np
import pandas as pd
import requests
import urllib.request
from urllib.request import urlopen
from bs4 import BeautifulSoup 
from lxml import etree
import json
import csv,codecs
import time

Notes:
- Find the ranking website, retrieve the trending tiktok links
- extract the top comments.

In [131]:
def getUrls(n):
    url= "http://www.ijiandao.com/hot/media/douyin"
    headers = { 
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36'
    } 
    try:
        response = requests.get(url,headers=headers)
        soup = BeautifulSoup(response.text,'lxml')
    except Exception as e:
        print(e)
    links = soup.find(class_ = "hot_new_list hot_classify").findAll("a")
    urls = []
    txt = []
    for i in range(n):
        link = links[i]
        u = "http://www.ijiandao.com" + link['href']
        title = link.getText()
        urls.append(u)
        txt.append(title)
    df = pd.DataFrame({'url':urls,'title':txt})
    return df

In [158]:
def get_tiktokLink(df,num_vid):
    ret = []
    urls = df['url']
    for url in urls:
        headers = { 
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36'
        } 
        try:
            response = requests.get(url,headers=headers)

            #将一段文档传入BeautifulSoup的构造方法,就能得到一个文档的对象, 可以传入一段字符串
            soup = BeautifulSoup(response.text,'lxml')
        except Exception as e:
            print(e)
        itm = soup.find(class_ = "hot_link")['href']
        ret.append(itm)
    return ret

In [165]:
def get_tiktokComment(num_vid):
    if (num_vid > 50):
        print("number entered exceeded 50")
        return
    f = codecs.open('tiktok_commments.csv','w','utf_8_sig')
    word_writer = csv.writer(f)
    word_writer.writerow(["comment","vid_title","vid_url"])
    df = getUrls(num_vid)
    urls = get_tiktokLink(df, num_vid)
    tot = 0
    for i in range(len(df)):
        title = df.iloc[i].title
        link = urls[i]
        headers = { 
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36'
        } 
        try:
            response = requests.get(link,headers=headers)
            soup = BeautifulSoup(response.text,'lxml')
        except Exception as e:
            print(e)
        comments = soup.getText().split("...")[1:-1]
        l = 0
        for com in comments:
            txt = com.split("展开")[0]
            word_writer.writerow([txt,title,url])
            l+=1
        print(title, " finished. ", str(l), " comments added")
        tot +=l
    return tot

In [166]:
def getTikTok():
    num_vid = int(input("how many videos would you like to crawl?(max 50)"))
    try:
        tot = get_tiktokComment(num_vid)
    except Exception as e:
        print("some error has occured.")
        print(e)
    print("ALL FINISHED! ",num_vid, " added; ",tot, " comments added in total." )

In [167]:
getTikTok()

how many videos would you like to crawl?(max 50)2
全红婵才14岁  finished.  19  comments added
张怡宁回应不让女儿玩银牌  finished.  20  comments added
ALL FINISHED!  2  added;  39  comments added in total.
